## Upload default computate Solr configset
A Solr configset is a data schema for a search engine. It specifies the exact fields and their data types that are available in the search engine to use.

In [ ]:
oc exec pod/solr-0 -- \
  install -d /bitnami/solr/backup/
oc rsync solr/ pod/solr-0:/bitnami/solr/backup/
echo DONE

In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)
oc exec pod/solr-0 -- \
  bash -c 'curl -s -X POST -u "$SOLR_ADMIN_USERNAME:$SOLR_ADMIN_PASSWORD" \
      -H "Content-Type:application/octet-stream" \
      --data-binary @/bitnami/solr/backup/computate-configset.zip \
      "http://localhost:8983/solr/admin/configs?action=UPLOAD&name=computate&overwrite=true"'
echo DONE

In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)
oc exec pod/solr-0 -- \
  bash -c 'export SOLR_COLLECTION='"$SOLR_COLLECTION"' && SOLR_AUTH_TYPE=basic SOLR_AUTHENTICATION_OPTS="-Dbasicauth=$SOLR_ADMIN_USERNAME:$SOLR_ADMIN_PASSWORD" /opt/bitnami/solr/bin/solr create_collection -c $SOLR_COLLECTION -n computate'
echo DONE

## Run SQL create scripts with new PostgreSQL tables
When we create new Smart Data Models that are persisted in the PostgreSQL database, we need to make sure that our database has all the tables and fields required to persist the data. Run the commands below to rsync the `db-create.sql` script to the PostgreSQL pod. It will connect again and apply all of the database schema changes to the database. 

In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)
oc exec pod/postgresql-0 -- mkdir -p /bitnami/postgresql/sql/
oc rsync $SITE_SRC/src/main/resources/sql/ \
  pod/postgresql-0:/bitnami/postgresql/sql/
oc exec pod/postgresql-0 -- \
  env PGPASSWORD=$(oc get secret/postgres-pguser-$DATABASE_USERNAME -o jsonpath={.data.password} | base64 -d) \
  psql -U $DATABASE_USERNAME -d $DATABASE_DATABASE -f /bitnami/postgresql/sql/db-create.sql
echo DONE

# Deploy your site! 

At this point, you have built your new site, containerized it, 
and created the kustomization infrastructure as code to deploy it. 


In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)
echo $SITE_HOST_NAME
envsubst < $OVERLAY_SITE_PATH/deployment.yaml | grep SITE_HOST_NAME -A 1

In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)
replace '$SITE_HOST_NAME' "$SITE_HOST_NAME" -- $OVERLAY_SITE_PATH/deployment.yaml
replace '$SITE_BASE_URL' "$SITE_BASE_URL" -- $OVERLAY_SITE_PATH/deployment.yaml
replace '$SITE_HOST_NAME' "$SITE_HOST_NAME" -- $OVERLAY_SITE_PATH/route.yaml

In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)
oc apply -k $OVERLAY_SITE_PATH